In [150]:
# pip install beautifulsoup4
# !pip install requests
# !pip install mysql-connector-python


In [1]:
import requests
from bs4 import BeautifulSoup
from collections import deque
from urllib.parse import urljoin
import json
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from tqdm import tqdm
nltk.download('wordnet')
nltk.download('omw')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [18]:
url_regex = re.compile(
r'^(?:http|https):\/\/',
re.IGNORECASE
)

url = 'https://solarsystem.nasa.gov/planets/overview/'
max_depth = 10
data = {}
content = []

res = requests.get(url)
soup_queue = BeautifulSoup(res.content, 'html.parser')

# Inicia a fila com o elemento raiz
queue = deque([soup_queue])

# Enquanto houver elementos na fila
while queue:
    # Remove o próximo elemento da fila
    current_element= queue.popleft()
    
    # Procura todos os links no elemento atual
    links = current_element.find_all('a')
    
    # Adiciona todos os elementos encontrados na fila
    for link in links:
        href = link.get('href')
            
        # Verifica se o link é diferente de None e começa com "https" ou "http"
        if href and (href.startswith('https') or href.startswith('http')):
            # Se já começar com "https" ou "http", utiliza o link sem modificação
            final_link = href
        else:
            # Se não começar com "https" ou "http", adiciona "https://" antes do link
            final_link = urljoin(url, href)
            print(final_link)

        if url_regex.match(final_link):
            valid_link = final_link
        


https://solarsystem.nasa.gov/
http
https://solarsystem.nasa.gov/
http
https://solarsystem.nasa.gov/
http
https://solarsystem.nasa.gov/planets/overview/#page
http
javascript:void(0);
http
https://solarsystem.nasa.gov/planets/overview/
http
https://solarsystem.nasa.gov/planets/overview
http
https://solarsystem.nasa.gov/planets/in-depth
http
https://solarsystem.nasa.gov/planets/planetary-analogs
http
https://solarsystem.nasa.gov/planets/analog-explorer
http
https://solarsystem.nasa.gov/solar-system/sun/overview/
http
https://solarsystem.nasa.gov/planets/mercury/overview/
http
https://solarsystem.nasa.gov/planets/venus/overview/
http
https://solarsystem.nasa.gov/planets/earth/overview/
http
https://solarsystem.nasa.gov/planets/mars/overview/
http
http
https://solarsystem.nasa.gov/planets/jupiter/overview/
http
https://solarsystem.nasa.gov/planets/saturn/overview/
http
https://solarsystem.nasa.gov/planets/uranus/overview/
http
https://solarsystem.nasa.gov/planets/neptune/overview/
http
http

In [4]:
def webscrap(input_url, max_depth):

    url_regex = re.compile(
    r'^(?:http|https):\/\/',
    re.IGNORECASE
    )

    # Define a página web a ser analisada
    url = input_url
    max_depth = max_depth
    data = {}
    content = []

    res = requests.get(url)
    soup_queue = BeautifulSoup(res.content, 'html.parser')

    # Inicia a fila com o elemento raiz
    queue = deque([soup_queue])

    # Enquanto houver elementos na fila
    while queue:
        # Remove o próximo elemento da fila
        current_element= queue.popleft()
        
        # Procura todos os links no elemento atual
        links = current_element.find_all('a')
        
        # Adiciona todos os elementos encontrados na fila
        for link in links:
            print(links)
            # Obtem o valor do atributo href do link
            href = link.get('href')
            
            # Verifica se o link é diferente de None e começa com "https" ou "http"
            if href and (href.startswith('https') or href.startswith('http')):
                # Se já começar com "https" ou "http", utiliza o link sem modificação
                final_link = href
            else:
                # Se não começar com "https" ou "http", adiciona "https://" antes do link
                final_link = urljoin(url, href)

            if url_regex.match(final_link):
                valid_link = final_link
                #list_link.append(valid_link)
            
            # Faz a requisição do conteúdo da página
            response = requests.get(valid_link)

            # Cria o objeto Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Obtém o título da página
            title = soup.title.string
            #title = valid_link.replace("https://", "").replace("\\", "").replace("https:\\", "").replace("www", "")

            # Obtém o conteúdo da página
            p_tags = soup.find_all('p')
            for p in p_tags:
                content.append(p.text)
            join_content = ' '.join(content)

            # Cria um dicionário com o título e o conteúdo da página
            data = {"title": title, "content": re.sub(r'\r?\n|\r', '', join_content), "url": valid_link}

            # # Cria um arquivo JSON com o nome do link e o conteúdo do dicionário
            # Nome do diretório a ser criado
            dir_name = str(re.sub(r'\W+', '_', url))

            # Verifica se o diretório já existe
            if not os.path.exists(dir_name):
                # Cria o diretório
                os.makedirs(dir_name)

        
            clean_file_name = re.sub(r'\W+', '_', title.replace("https", ""))
            file_name = os.path.join(dir_name,f"{clean_file_name}.json")
            file_path = os.path.join(dir_name, file_name)
            if not os.path.exists(file_path):
                #os.makedirs(file_path)
                with open(file_name, "w") as f:
                    json.dump(data, f)
    
    content_list = []
    title_list = []
    url_list = []

    for filename in os.listdir(dir_name):
        file_path_ = os.path.join(dir_name, filename)
        with open(file_path_, "r") as file:
            data = json.load(file)
            data_content = str(data['content'])
            content_list.append(data_content)
            title_list.append(data['title'])
            url_list.append([data['url']][0])
    data_dict={'Title':title_list,'Content':content_list,'Url':url_list}
    df = pd.DataFrame(data_dict)

    return df

In [5]:
webscrap_df = webscrap('https://solarsystem.nasa.gov/planets/overview/', 15)
webscrap_df

[<a class="nasa_logo" href="/" title="Home">NASA</a>, <a class="top_logo" href="/" title="Home">Solar System Exploration</a>, <a class="sub_logo" href="/" title="Home">Our Galactic Neighborhood</a>, <a class="visuallyhidden focusable" href="#page">Skip Navigation</a>, <a class="menu_button" href="javascript:void(0);" id="menu_button">
<span class="menu_icon">
menu
</span>
</a>, <a class="modal_close" id="modal_close">
<span class="modal_close_icon">
close modal
</span>
</a>, <a href="/planets/overview">Overview</a>, <a href="/planets/in-depth">What is a Planet?</a>, <a href="/planets/planetary-analogs">Planetary Analogs</a>, <a href="/planets/analog-explorer">Analog Explorer</a>, <a href="/solar-system/sun/overview/">our star</a>, <a href="/planets/mercury/overview/">Mercury</a>, <a href="/planets/venus/overview/">Venus</a>, <a href="/planets/earth/overview/">Earth</a>, <a href="/planets/mars/overview/">Mars</a>, <a href="https://www.nasa.gov/press-release/touchdown-nasas-mars-persever

,Title,Content,Url
0,Analog Explorer | Planets – NASA Solar System ...,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/planets/analog-ex...
1,Home – NASA Solar System Exploration,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/
2,Overview | Earth – NASA Solar System Exploration,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/planets/earth/ove...
3,Overview | Mars – NASA Solar System Exploration,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/planets/mars/over...
4,Overview | Mercury – NASA Solar System Explora...,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/planets/mercury/o...
5,Overview | Planets – NASA Solar System Explora...,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/planets/overview
6,Overview | Sun – NASA Solar System Exploration,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/solar-system/sun/...
7,Overview | Venus – NASA Solar System Exploration,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/planets/venus/ove...
8,Planetary Analogs | Planets – NASA Solar Syste...,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/planets/planetary...
9,What is a Planet? | Planets – NASA Solar Syste...,Dana Bolles manages the day-to-day operations ...,https://solarsystem.nasa.gov/planets/in-depth


In [154]:
def create_index(df):
    doc_content = df['Content']

    # Define a list of stopwords
    my_stopwords = stopwords.words('english')

    # Initialize TfidfVectorizer with custom stopword list
    vectorizer = TfidfVectorizer(stop_words=my_stopwords)
    tfidf = vectorizer.fit_transform(doc_content).todense()

    indice_invertido = {}
    for term in tqdm(vectorizer.vocabulary_):
        dic_score = {}
        for i, doc in enumerate(doc_content):
            j = vectorizer.vocabulary_[term] #term frequency
            value_tfidf = tfidf[i,j]
            #print(float(value_tfidf))
            if value_tfidf > 0:
                dic_score[df['Url'][i]] = value_tfidf
        indice_invertido[term] = dic_score
    
    return indice_invertido

In [164]:
inv_index = create_index(webscrap_df)

100%|██████████| 1815/1815 [00:00<00:00, 7100.30it/s]


In [364]:
def buscar_inv(palavras, indice):
    words = palavras.split()
    assert type(words)==list
    resultado = dict()
    highest_score = []
    for p in words:
        if p in indice.keys():
            for documento in indice[p].keys():
                if documento not in resultado.keys():
                    resultado[documento] = indice[p][documento]
                else:
                    resultado[documento] += indice[p][documento]
            
        else:
            resultado = 'Perdão, não encontrei o que está buscando.'

    return resultado

In [365]:
resultado = buscar_inv('cold',inv_index)

In [160]:
if isinstance(resultado, str):
    print(resultado)

In [366]:
max_value_url = max(resultado, key=lambda x: resultado[x])
title = webscrap_df[webscrap_df['Url'] == max_value_url]['Title'].values[0]
content = webscrap_df[webscrap_df['Url'] == max_value_url]['Content'].values[0]

print(title)
print(content)
print(max_value_url)
print("Score:", resultado[max_value_url])

Overview | Neptune – NASA Solar System Exploration
Dana Bolles manages the day-to-day operations of the science.nasa.gov website. Let NASA be your guide to the night sky. Dr. Nicola Fox is the new associate administrator for the agency’s Science Mission Directorate. U.S. Poet Laureate Ada Limón, known for work that explores the human connection to the natural world, is crafting a new poem dedicated to the Europa Clipper mission. Dr. Lori Glaze is NASA's director of planetary science. "I remember looking at that picture and thinking that there is something weird going on here and that it has important implications for the whole history of the planet, its climate, its atmosphere, its geology." - David Grinspoon, scientist Science Writers: Phil Davis & Steve CarneySocial Media Lead: Bill DunfordDesign & Development: Moore Boeck Site last updated: March 27, 2023 Dana Bolles manages the day-to-day operations of the science.nasa.gov website. Let NASA be your guide to the night sky. Dr. Nicol

In [367]:
def get_synonyms(palavra,df):
    words = palavra

    vectorizer = TfidfVectorizer(stop_words=my_stopwords)
    tfidf = vectorizer.fit_transform(df['Content']).todense()

    highest_score = 0
    syns1 = wordnet.synsets(words)[0]
    seen_words = set()  # Set to keep track of words already in the list
    
    for term in vectorizer.vocabulary_:
        try:
            syns2 = wordnet.synsets(str(term))
            for syn in syns2:
                syn_name = syn.name().split('.')[0]
                if syn_name in vectorizer.vocabulary_:
                    #print(syn_name)
                    score = syns1.wup_similarity(syns2)
                    if score != 1:  # Exclude perfect matches
                        word = syns2.name().split('.')[0]
                        if word not in seen_words:  # Append only if word is not already in list
                            if score > highest_score:
                                highest_score = score
                                highest_word = word
                                seen_words.add(word)
        except IndexError:
            continue

    return highest_word

In [368]:
def get_synonyms(palavra,df):
    words = palavra

    vectorizer = TfidfVectorizer(stop_words=my_stopwords)
    tfidf = vectorizer.fit_transform(df['Content']).todense()

    highest_score = 0
    syns1 = wordnet.synsets(words)[0]
    seen_words = set()  # Set to keep track of words already in the list
    
    for term in vectorizer.vocabulary_:
        try:
            syns2 = wordnet.synsets(str(term))
            for syn in syns2:
                syn_name = syn.name().split('.')[0]
                if syn_name in vectorizer.vocabulary_:
                    score = syns1.wup_similarity(syn)
                    if score is not None and score != 1:  # Exclude perfect matches
                        word = syn_name
                        if word not in seen_words:  # Append only if word is not already in list
                            if score > highest_score:
                                highest_score = score
                                highest_word = word
                                seen_words.add(word)
        except IndexError:
            continue

    return highest_word


In [369]:
def multiple_synonyms(palavras,df):
    vectorizer = TfidfVectorizer(stop_words=my_stopwords)
    tfidf = vectorizer.fit_transform(df['Content']).todense()

    words = palavras.split()
    assert type(words)==list

    for i in range(len(words)):
        try:
            if words[i] not in vectorizer.vocabulary_:
                syn = get_synonyms(words[i],webscrap_df)
                words[i] = syn
        except IndexError:
            continue
    sentence = " ".join(words)

    return sentence

In [370]:
multiple_synonyms('big elephant green planet',webscrap_df)

'big fox red planet'

In [371]:
buscar_inv('big fox red planet',inv_index)

{'https://solarsystem.nasa.gov/planets/analog-explorer': 0.29097540577059083,
 'https://solarsystem.nasa.gov/planets/earth/overview/': 0.3237703734959119,
 'https://solarsystem.nasa.gov/planets/jupiter/overview/': 0.364339782941639,
 'https://solarsystem.nasa.gov/planets/mars/overview/': 0.3556399730978003,
 'https://solarsystem.nasa.gov/planets/mercury/overview/': 0.2973650455686902,
 'https://solarsystem.nasa.gov/planets/neptune/overview/': 0.3676215775083026,
 'https://solarsystem.nasa.gov/planets/saturn/overview/': 0.3654051248890614,
 'https://solarsystem.nasa.gov/solar-system/sun/overview/': 0.2505919110600185,
 'https://solarsystem.nasa.gov/planets/uranus/overview/': 0.3690014169428351,
 'https://solarsystem.nasa.gov/planets/venus/overview/': 0.303987067949247,
 'https://solarsystem.nasa.gov/planets/planetary-analogs': 0.29628999107146875,
 'https://www.nasa.gov/press-release/touchdown-nasas-mars-perseverance-rover-safely-lands-on-red-planet': 0.3556399730978003,
 'https://solar